In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [24]:
data = pd.read_csv('profiling_gini_before_changes.csv.csv')

In [25]:
def transform_milisec(sec):
    try:
        time = float(sec[:-2])
        unit = sec[-2:]
        if unit == 'ms':
            return time
        elif unit == 'ns':
            return time/1000000
        elif unit == 'µs':
            return time/1000
        else:
            return time
    except:
        print(sec[:-1])
        time = float(sec[:-1])
        
        return time*1000

In [33]:
total_time = data.iloc[-1]['function']
data = data.iloc[:-1]
total_time

'7.826838786s'

In [34]:
data

,function,time,len
0,unique_vals,10.925471ms,9000.0
1,Gini,3.820273ms,9000.0
2,unique_vals,77.717µs,62.0
3,Gini,5.648µs,62.0
4,unique_vals,19.270898ms,8938.0
...,...,...,...
3379,Gini,114.88µs,2602.0
3380,unique_vals,1.552378ms,2441.0
3381,Gini,88.384µs,2441.0
3382,unique_vals,101.822µs,161.0


In [35]:
data['time'] = data['time'].apply(transform_milisec)

/var/folders/vf/q4kws5qs0vjgrbd6tsb44wfr0000gn/T/ipykernel_76547/3931296818.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['time'] = data['time'].apply(transform_milisec)


In [40]:
data

,function,time,len
0,unique_vals,10.925471,9000.0
1,Gini,3.820273,9000.0
2,unique_vals,0.077717,62.0
3,Gini,0.005648,62.0
4,unique_vals,19.270898,8938.0
...,...,...,...
3379,Gini,0.114880,2602.0
3380,unique_vals,1.552378,2441.0
3381,Gini,0.088384,2441.0
3382,unique_vals,0.101822,161.0


In [39]:
data['time'].sum()/1000

4.74788653